<h1>Motivation</h1>
<p>This notebook is designed to analyze the description of every injury that is recorded as Sprain/strain InjuryIllnessType to see if there are some trends that we can find.</p>

# Import the libraris to read and manipulate data.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

In [3]:
data = pd.read_excel('strain-sprints data.xlsx')
data.head()

,IICase_ID,DistinctIICaseID,WorldRegion,Orgname,SubOrg,Site,CaseDate,CaseType,CaseLevel,InjuryIllnessType,PrincipalBodyPart,DetailedBodyPart,GensuiteURL,Description
0,3109,163441,U.S.,North America Fulfillment Centers (FCs),Legacy Sortable,ABE2,2017-01-21,Injury,Level C,Sprain/strain,Shoulder,Right Shoulder,https://amazon.gensuite.com/amazon/ehs/measure...,"On January 21, 2016, AA was tasked to stow. AA..."
1,3108,163440,U.S.,North America Fulfillment Centers (FCs),Legacy Sortable,ABE2,2017-01-22,Injury,Level C,Sprain/strain,Knee,Right Knee,https://amazon.gensuite.com/amazon/ehs/measure...,Description: On 1/22/17 at approximately 22:30...
2,3152,171762,U.S.,North America Fulfillment Centers (FCs),Legacy Sortable,ABE2,2017-01-30,Injury,Level C,Sprain/strain,Wrist,Right Wrist,https://amazon.gensuite.com/amazon/ehs/measure...,"On 3/16/2017 at 02:30am, a Stow AA reported pa..."
3,3150,171747,U.S.,North America Fulfillment Centers (FCs),Legacy Sortable,ABE2,2017-03-09,Injury,Level C,Sprain/strain,Back,Lower Back,https://amazon.gensuite.com/amazon/ehs/measure...,"On 3/15/17 at 10:40 PM, AA with hire date of 2..."
4,3167,177736,U.S.,North America Fulfillment Centers (FCs),Legacy Sortable,ABE2,2017-03-26,Injury,Level C,Sprain/strain,Wrist,Left Wrist,https://amazon.gensuite.com/amazon/ehs/measure...,AA with DOH of 10/27/2016 and Pick TWI trainin...


In [4]:
data.shape

(10340, 14)

## Time to preprocess the text. 
<p>I will use the libraries __`spaCy`__ and __`re`__ for this purpose.</p>


In [5]:
import spacy
nlp = spacy.load('en')

C:\Users\ndsamuel\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\ndsamuel\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
C:\Users\ndsamuel\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\ndsamuel\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


In [6]:
#prevent DeprecationWarnings RuntimeWarnings and UserWarnings from showing
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=RuntimeWarning)
warnings.filterwarnings("ignore",category=UserWarning)

## Remove recurring words
<p> Due to the way I defined the functions to replace the dates and times in each Description with the words date and time respectively, there is going to be many occurrectes of the words date date. I will define a function to remove repeated words and strips out any multiple white spaces.

In [7]:
def remove_recurring_words(sent):
    tokens = sent.split()
    return ' '.join([word for index, word in enumerate(tokens) if not tokens[index] == tokens[index - 1]])
remove_recurring_words('here he is saying saying   all men men are are equal equal')

'here he is saying all men are equal'

In [8]:
data['Description'] = data['Description'].apply(remove_recurring_words)

## Using spaCy to tokenize the Text.

<p>We remove all punctuation marks, adverbs, pronouns, and definite and indefinite articles form the from every description</p>

<p> Punctuation marks make sense to humans but in many machine learning algorithms, there are not helpful. I will take the out together with any special symbol.</p>

In [9]:
import re
data['RawPrep'] = data.Description.apply(lambda desc: [word.text.lower() for word in nlp(desc) if word.pos_ not in [ 'ADV', 'ADP', 'DET', 'PART', 'PRON', 'PUNCT']])

## Removing Stop-Words:
Stopwords are words that occour too frequently in a text corpus but which do not carry a lot of meaning. I will use the set of stopwords from `nltk`. Also, since aa in our corpus represents the word associate, I will replace all occurreces of the word aa with the word associate.

<p> I will also remove words with less than 3 characters.

In [10]:
from nltk.corpus import stopwords
my_stopwords = stopwords.words('english')
print(my_stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [13]:
data['RawPrep1'] = data['RawPrep'].apply(lambda x: [word for word in x if ((len(word) > 2 and word not in my_stopwords) )])

In [14]:
data.head(2)

,IICase_ID,DistinctIICaseID,WorldRegion,Orgname,SubOrg,Site,CaseDate,CaseType,CaseLevel,InjuryIllnessType,PrincipalBodyPart,DetailedBodyPart,GensuiteURL,Description,RawPrep,RawPrep1
0,3109,163441,U.S.,North America Fulfillment Centers (FCs),Legacy Sortable,ABE2,2017-01-21,Injury,Level C,Sprain/strain,Shoulder,Right Shoulder,https://amazon.gensuite.com/amazon/ehs/measure...,"On January 21, 2016, AA was tasked to stow. AA...","[january, 21, 2016, aa, was, tasked, stow, aa,...","[january, 2016, tasked, stow, attended, stretc..."
1,3108,163440,U.S.,North America Fulfillment Centers (FCs),Legacy Sortable,ABE2,2017-01-22,Injury,Level C,Sprain/strain,Knee,Right Knee,https://amazon.gensuite.com/amazon/ehs/measure...,Description: On 1/22/17 at approximately 22:30...,"[description, 1/22/17, 22:30, amm, was, notifi...","[description, 1/22/17, 22:30, amm, notified, m..."


## Time to perform some Topic modelling.
Let me guess that the number of topics in these documents (descriptions) is equal tot he number of principal body part.

In [28]:
data.PrincipalBodyPart.nunique()

24

## Topic modelling - GENSIM Library.
I first import the library and prepare a corpus dictionary, then I create a bag of words and apply the lda model on the bag of words.

 <h3> The steps to create a gensim topic model are:</h3>
 <ul>
    <li>Create a corpus dictionary that stores words together with their indexes using the `Dictionary` method of `gensim.corpora` module.</li>
    <li>Store the list of tokenized documents. Since each tokenized document is a list of words, you have a list of list of words.</li>
    <li>Create a count corpus that is a list of lists of tuples (word indexes, frequecy) for each word in the document bag of words.</li>
</ul>

In [29]:
import gensim.corpora as corpora
import gensim
from gensim.models import CoherenceModel
# Create Dictionary
id2word = corpora.Dictionary(data.RawPrep1.values.tolist())

# Create Corpus
texts = data.RawPrep1.values.tolist()

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View the first document corpus
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 4), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 3), (24, 3), (25, 2), (26, 1), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 4), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 2), (57, 3), (58, 3), (59, 1), (60, 7), (61, 1), (62, 2), (63, 1), (64, 3), (65, 2), (66, 1), (67, 2), (68, 1), (69, 1), (70, 2), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1)]]


In [30]:
id2word[0]

'2/10'

Sometimes, it is really important to have human readable format of corpus (word, frequency). We check the (word, frequency) corpus for the first document below.

In [31]:
print([[(id2word[ID], freq) for ID, freq in cp] for cp in corpus[:1]])

[[('2/10', 1), ('2016', 1), ('2017', 1), ('4-days', 1), ('6/10', 2), ('amazon', 1), ('amcare', 1), ('amount', 1), ('appropriate', 1), ('arm', 1), ('asked', 1), ('attended', 1), ('attention', 1), ('better', 1), ('bins', 4), ('causes', 1), ('diverter', 1), ('get', 1), ('got', 1), ('higher', 1), ('hoping', 1), ('hurting', 1), ('icing', 1), ('injury', 3), ('items', 3), ('january', 2), ('knew', 1), ('level', 1), ('lost', 1), ('lunch', 2), ('making', 1), ('may', 1), ('medical', 1), ('method', 1), ('mods', 1), ('months', 1), ('needed', 1), ('night', 1), ('noted', 1), ('one', 1), ('overstretched', 1), ('pain', 4), ('pcp', 2), ('point', 1), ('post', 1), ('prescribed', 1), ('proceeded', 1), ('pulled', 1), ('racks', 1), ('reaching', 1), ('recordable', 1), ('repetitive', 1), ('reported', 2), ('reporting', 1), ('right', 1), ('saw', 1), ('shift', 2), ('shoulder', 3), ('small', 3), ('started', 1), ('stated', 7), ('states', 1), ('stow', 2), ('stowed', 1), ('stowing', 3), ('stretches', 2), ('tasked', 1

# Fit a topic model and visualize the topics
My favorite visualization for topic models is `pyLDAvis.` Make sure to import pyLDAvis.gensim as well.

At this point, it seems as if there are 3 main topics in our documents but there seems to be 6 other topics that are not being accurately captured.

Also, I don't really like deprecations warnings so I will __ignore__ those.

In [57]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [18]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # This import is neccessary for pyLDAvis to be used in python.
import matplotlib.pyplot as plt


In [20]:
#prevent DeprecationWarnings RuntimeWarnings and UserWarnings from showing
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=RuntimeWarning)
warnings.filterwarnings("ignore",category=UserWarning)

In [33]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=24,
                                           chunksize=2,
                                           passes = 5)                                  
#                                            random_state=100,
#                                            update_every=100,
#                                            alpha='auto',
#                                            per_word_topics=True)
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus =corpus, dictionary=id2word)
vis

C:\Users\ndsamuel\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.390948 -0.026571       1        1  14.921101
8     -0.373219 -0.091289       2        1  14.071115
19    -0.032305  0.295571       3        1   6.493378
23     0.054951 -0.145547       4        1   4.534355
12    -0.009450 -0.014865       5        1   4.341656
16    -0.023772  0.265909       6        1   4.183816
1      0.057081 -0.138432       7        1   4.052224
2     -0.027919  0.006431       8        1   3.973800
3      0.025910 -0.006407       9        1   3.500003
4      0.060559 -0.035048      10        1   3.373532
5      0.026123 -0.002240      11        1   3.341238
14     0.040210 -0.032000      12        1   3.325528
0      0.026483  0.014475      13        1   3.246007
7      0.032972  0.022486      14        1   3.206656
10     0.035317  0.030921      15        1   3.118575
20     0.041264  0.161447      16        1   2.857546
17     0.047747 -0.060351      17        1   2.710546
15     0.064301 -0.087666      18        1   2.536001
6      0.050132 -0.025726      19        1   2.327518
11     0.049030 -0.016311      20        1   2.116429
22     0.054408 -0.028254      21        1   2.070374
13     0.069040 -0.058623      22        1   2.018256
21     0.060901 -0.017378      23        1   1.976000
9      0.061187 -0.010532      24        1   1.704341, topic_info=      Category          Freq           Term         Total  loglift  logprob
term                                                                       
90     Default  20188.000000      associate  20188.000000  30.0000  30.0000
73     Default  11230.000000           work  11230.000000  29.0000  29.0000
41     Default  26648.000000           pain  26648.000000  28.0000  28.0000
115    Default  12709.000000       incident  12709.000000  27.0000  27.0000
23     Default  13049.000000         injury  13049.000000  26.0000  26.0000
778    Default   5465.000000         pallet   5465.000000  25.0000  25.0000
386    Default   5632.000000           site   5632.000000  24.0000  24.0000
6      Default  10130.000000         amcare  10130.000000  23.0000  23.0000
334    Default   7044.000000           left   7044.000000  22.0000  22.0000
60     Default   9149.000000         stated   9149.000000  21.0000  21.0000
343    Default   5124.000000        process   5124.000000  20.0000  20.0000
52     Default  11225.000000       reported  11225.000000  19.0000  19.0000
32     Default   4100.000000        medical   4100.000000  18.0000  18.0000
194    Default   3876.000000  investigation   3876.000000  17.0000  17.0000
348    Default   4828.000000         safety   4828.000000  16.0000  16.0000
374    Default   3687.000000        lifting   3687.000000  15.0000  15.0000
629    Default   4570.000000            box   4570.000000  14.0000  14.0000
246    Default   4094.000000           back   4094.000000  13.0000  13.0000
271    Default   3337.000000           pick   3337.000000  12.0000  12.0000
290    Default   3664.000000         strain   3664.000000  11.0000  11.0000
72     Default   3381.000000           went   3381.000000  10.0000  10.0000
117    Default   3242.000000           knee   3242.000000   9.0000   9.0000
27     Default   6783.000000          level   6783.000000   8.0000   8.0000
5      Default   3296.000000         amazon   3296.000000   7.0000   7.0000
280    Default   5159.000000   restrictions   5159.000000   6.0000   6.0000
308    Default   2866.000000           cart   2866.000000   5.0000   5.0000
237    Default   2649.000000          wrist   2649.000000   4.0000   4.0000
630    Default   2779.000000          boxes   2779.000000   3.0000   3.0000
326    Default   2896.000000           hand   2896.000000   2.0000   2.0000
698    Default   5680.000000           area   5680.000000   1.0000   1.0000
...        ...           ...            ...           ...      ...      ...
1368   Topic24    656.536926           lane  

#### Print the top 20 Keyword in each of the 24 topics with the importance of each word.

In [34]:
import pprint
for topic in lda_model.show_topics(num_words=30, num_topics = 24, formatted = False):
    print('Topic {}:'.format(topic[0] +1), '\n', ' | '.join([wordpair[0] for wordpair in topic[1]]), '\n', '--'*78)
#Equivallent to pprint(lda_model.show_topics(num_words=20, formatted = False))

Topic 1: 
 investigation | physical | related | resulting | sought | occupational | high | freezer | provide | specific | complete | walk | weeks | stating | carrying | soreness | non | home | evaluated | clinic | response | documentation | standing | told | risk | come | note | allow | improper | marked 
 ------------------------------------------------------------------------------------------------------------------------------------------------------------
Topic 2: 
 left | side | floor | sort | causing | foot | ground | fall | rest | fell | falling | took | trying | leg | september | spill | slipped | stack | orders | help | make | space | walked | tripped | plastic | hit | sit | balance | fresh | june 
 ------------------------------------------------------------------------------------------------------------------------------------------------------------
Topic 3: 
 associate | medication | notified | muscle | room | area | assistance | bags | conservative | partner | arrival |

 injury | pain | amcare | stated | reported | treatment | right | recordable | report | osha | making | diagnosed | working | could | time | received | days | given | reporting | aid | provider | requested | see | felt | treated | escorted | initial | physician | ice | follow 
 ------------------------------------------------------------------------------------------------------------------------------------------------------------
Topic 20: 
 box | found | case | item | restrictions | working | place | doctor | bin | outside | rack | completed | attempted | product | twisting | put | shelf | twisted | made | able | minutes | required | paperwork | brought | order | racks | neck | visited | bakers | hired 
 ------------------------------------------------------------------------------------------------------------------------------------------------------------
Topic 21: 
 pick | picking | associates | month | seen | restrictions | end | feet | return | walking | importance | caused | 

<h1>How to inteprete the topics:</h1>

<p> Each topic is made of key words or __sailient terms__ which are the words that carry the most meaning in that topic. The number besides each word is a measure of the importance of the word to that topic. These numbers are also called the __weights__ and they follow a probability distribution on the topic and so the sum of the weights for each topic is unity.</p>
<p> The topics are not printed in any order.</p>
<p> We use the topic coherence and perplexity to determin which how good our topic model is. I have printed the coherence and perplexity scores for our topic model below.</p>

In [35]:
# Compute Perplexity """lower the better."""
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  

# Compute Coherence Score  """ Higher is better """
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v', topn=30)
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print("Topics_Coherence, Standard_Deviation, Support")# support is the topn # of words in the topic
coherence_model_lda.get_coherence_per_topic(with_std=True, with_support=True)



Perplexity:  -9.162445666129194

Coherence Score:  0.27971379970338855
Topics_Coherence, Standard_Deviation, Support


[(0.27652524758626484, 0.19014347156800823, 30),
 (0.3556913437448355, 0.3256360302755269, 30),
 (0.20692983066594328, 0.13768482488415615, 30),
 (0.279614311577285, 0.12685501318486775, 30),
 (0.3214592516466578, 0.14763778352973964, 30),
 (0.2477303519044698, 0.20270473252673937, 30),
 (0.32122271142796954, 0.17207220237310944, 30),
 (0.20889114841896012, 0.20914158706500863, 30),
 (0.24692130496516415, 0.23703615609292572, 30),
 (0.22608336448012556, 0.14876131421984876, 30),
 (0.2405471547781272, 0.14663891182048347, 30),
 (0.3955398002629777, 0.13939809785324753, 30),
 (0.18594878405967077, 0.1706870826660491, 30),
 (0.4158218106690274, 0.15504677766609654, 30),
 (0.32383689479338545, 0.43561612990629967, 30),
 (0.23147536772293578, 0.13420014049888238, 30),
 (0.17061465883645982, 0.14827849077412472, 30),
 (0.25927166391154643, 0.16484680205233648, 30),
 (0.39771442037139676, 0.2766555054266788, 30),
 (0.22990311551589385, 0.15307054310078946, 30),
 (0.3274293602661448, 0.2099897

## It seems as if 24 was too large a number for the number of topics. Let's change the number to 15 and see if we get better results.

In [36]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15,
                                           random_state=100,
                                           chunksize=2,
                                           passes=10)#,
#                                            update_every=100,
#                                            alpha='auto',
#                                            per_word_topics=True)
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus =corpus, dictionary=id2word)
vis

C:\Users\ndsamuel\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.376288 -0.120364       1        1  32.129383
4     -0.028613  0.184157       2        1   8.305664
14    -0.165608  0.291124       3        1   7.503173
7     -0.138102 -0.140689       4        1   6.461175
8     -0.010188  0.111062       5        1   5.882811
9     -0.066610 -0.180592       6        1   5.544648
13     0.071039 -0.005629       7        1   5.371512
1      0.074534  0.000708       8        1   5.280529
10     0.040593  0.072555       9        1   4.002356
0      0.051252 -0.026268      10        1   3.961305
11     0.068810  0.012882      11        1   3.590982
5      0.114007 -0.059765      12        1   3.455698
12     0.109581 -0.041233      13        1   2.970207
3      0.142116 -0.063839      14        1   2.827039
6      0.113478 -0.034109      15        1   2.713523, topic_info=      Category          Freq          Term         Total  loglift  logprob
term                                                                      
90     Default  22384.000000     associate  22384.000000  30.0000  30.0000
73     Default  13400.000000          work  13400.000000  29.0000  29.0000
41     Default  24367.000000          pain  24367.000000  28.0000  28.0000
115    Default  10360.000000      incident  10360.000000  27.0000  27.0000
23     Default  12832.000000        injury  12832.000000  26.0000  26.0000
778    Default   4604.000000        pallet   4604.000000  25.0000  25.0000
32     Default   4318.000000       medical   4318.000000  24.0000  24.0000
629    Default   4590.000000           box   4590.000000  23.0000  23.0000
67     Default   6468.000000          time   6468.000000  22.0000  22.0000
52     Default  10550.000000      reported  10550.000000  21.0000  21.0000
6      Default  10539.000000        amcare  10539.000000  20.0000  20.0000
56     Default   5659.000000         shift   5659.000000  19.0000  19.0000
271    Default   4100.000000          pick   4100.000000  18.0000  18.0000
246    Default   4042.000000          back   4042.000000  17.0000  17.0000
60     Default   9572.000000        stated   9572.000000  16.0000  16.0000
290    Default   3652.000000        strain   3652.000000  15.0000  15.0000
57     Default   3057.000000      shoulder   3057.000000  14.0000  14.0000
280    Default   3952.000000  restrictions   3952.000000  13.0000  13.0000
27     Default   5695.000000         level   5695.000000  12.0000  12.0000
117    Default   3057.000000          knee   3057.000000  11.0000  11.0000
147    Default   2696.000000          team   2696.000000  10.0000  10.0000
99     Default   2675.000000         could   2675.000000   9.0000   9.0000
258    Default   3037.000000         found   3037.000000   8.0000   8.0000
386    Default   7574.000000          site   7574.000000   7.0000   7.0000
698    Default   4494.000000          area   4494.000000   6.0000   6.0000
53     Default   3317.000000     reporting   3317.000000   5.0000   5.0000
276    Default   2877.000000        proper   2877.000000   4.0000   4.0000
272    Default   2854.000000       picking   2854.000000   3.0000   3.0000
39     Default   2861.000000           one   2861.000000   2.0000   2.0000
195    Default   2709.000000          item   2709.000000   1.0000   1.0000
...        ...           ...           ...           ...      ...      ...
320    Topic15    574.220398      finished    575.045532   3.6055  -3.7467
113    Topic15    573.363708          form    574.188843   3.6055  -3.7482
1068   Topic15    509.908051     evaluated    510.733185   3.6053  -3.8655
1784   Topic15    462.980774          pole    463.805908   3.6051  -3.9620
368    Topic15    489.963013  compensation    490.854980   3.6051  -3.9054
4627   Topic15    427.340973         claim    428.166107   3.6050  -4.0421
3427   Topic15    416.128906      addition    416.954041   3.6049  -4.0687
1016   Topic15    409.617004          c

In [71]:
# Compute Perplexity """lower the better."""
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  

# Compute Coherence Score  """ Higher is better """
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v', topn=30)
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print("Topics_Coherence, Standard_Deviation, Support")# support is the topn # of words in the topic
coherence_model_lda.get_coherence_per_topic(with_std=True, with_support=True)


Perplexity:  -6.67482951565302

Coherence Score:  0.32315420021576635
Topics_Coherence, Standard_Deviation, Support


[(0.31258692239138774, 0.13821399015216862, 30),
 (0.3416852618528435, 0.2615573638246206, 30),
 (0.22771587347830993, 0.17602796581802269, 30),
 (0.2518312002836341, 0.16108268885590224, 30),
 (0.3075341341943814, 0.2995839565708181, 30),
 (0.5781199041746331, 0.3616204898370456, 30),
 (0.273709763339234, 0.19849957470297747, 30),
 (0.28665579093452914, 0.1560827374532069, 30),
 (0.46537561285439394, 0.21489312202950617, 30),
 (0.29811129735496955, 0.2630402012087099, 30),
 (0.2882796948734302, 0.11448399550415292, 30),
 (0.2629637766357751, 0.18137074664938527, 30),
 (0.2923081210010859, 0.1511520771709062, 30),
 (0.3557779971332776, 0.1602619094693828, 30),
 (0.3046576527346104, 0.14457716452667055, 30)]

From the look of the above model, it seems as if the words associate and incedent are skewing the distributions of the topics.

In [38]:
texts = [[word for word in doc if word not in ['pain', 'associate', 'incident', 'work', 'injury']] for doc in texts ]

In [39]:
id2word = corpora.Dictionary(data.RawPrep1.values.tolist())

# Create Corpus
texts = data.RawPrep1.values.tolist()

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View the first document corpus
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 4), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 3), (24, 3), (25, 2), (26, 1), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 4), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 2), (57, 3), (58, 3), (59, 1), (60, 7), (61, 1), (62, 2), (63, 1), (64, 3), (65, 2), (66, 1), (67, 2), (68, 1), (69, 1), (70, 2), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1)]]


Get the most probable topic for each document

In [49]:
l = []
for topic in lda_model.show_topics(num_words=30, num_topics=24, formatted = False):
    l.append(('Topic {}:'.format(topic[0] +1), ' | '.join([wordpair[0] for wordpair in topic[1]])))
#Equivallent to pprint(lda_model.show_topics(num_words=20, formatted = False))

In [42]:
topics = [lda_model.get_document_topics(doc_bow) for doc_bow in corpus]
topics

[[(0, 0.065769434),
  (2, 0.2250605),
  (3, 0.10844107),
  (4, 0.24928644),
  (5, 0.10712642),
  (6, 0.05033072),
  (7, 0.11441277),
  (10, 0.01895019),
  (13, 0.011088931),
  (14, 0.03738078)],
 [(0, 0.07345368),
  (1, 0.03513141),
  (2, 0.3213042),
  (3, 0.029088363),
  (4, 0.028683769),
  (5, 0.03945048),
  (6, 0.033127762),
  (7, 0.09694365),
  (9, 0.05607142),
  (11, 0.05162037),
  (12, 0.07925029),
  (13, 0.016660854),
  (14, 0.12261812)],
 [(0, 0.012359307),
  (1, 0.024014257),
  (2, 0.32012644),
  (4, 0.059216414),
  (5, 0.025045961),
  (6, 0.037967652),
  (8, 0.07601009),
  (10, 0.1849006),
  (13, 0.107450545),
  (14, 0.14264199)],
 [(1, 0.017787183),
  (2, 0.15213554),
  (3, 0.033685807),
  (4, 0.10565177),
  (5, 0.0189768),
  (6, 0.01874045),
  (7, 0.05296556),
  (8, 0.27217636),
  (9, 0.017413126),
  (10, 0.019475747),
  (13, 0.060582887),
  (14, 0.2201031)],
 [(1, 0.018544901),
  (2, 0.66437095),
  (3, 0.051952664),
  (4, 0.0189357),
  (6, 0.017205311),
  (7, 0.01146699),


Sort the topics in descending order so that the topic with the highest probability for each document shows up first and then get the topic Numbers.

In [43]:
sorted_topics = [sorted(doc_topics, key=lambda x: x[1], reverse =  True) for doc_topics in topics]
topic_num = [x[0][0] + 1  for x in sorted_topics]
print([(topic, topic_num.count(topic)) for topic in set(topic_num)])

[(1, 27), (2, 13), (3, 9716), (4, 14), (5, 186), (6, 8), (7, 29), (8, 54), (9, 32), (10, 83), (11, 17), (12, 17), (13, 60), (14, 31), (15, 53)]


In [51]:
data['PredictedTopic'] =['Topic {}'.format(topic) for index, topic in enumerate(topic_num)]

In [50]:
l

[('Topic 1:',
  'pallet | knee | associates | function | use | operations | attention | need | ladder | provide | asked | step | jack | waterspider | supply | handle | taken | push | way | chain | brace | fresh | assist | identified | noted | ups | tripped | refused | ladders | problem'),
 ('Topic 2:',
  'sent | line | scanner | break | job | sprain | follow | explained | position | continue | receive | called | station | belt | perform | reach | wanted | scheduled | keep | upper | orders | approached | 3/10 | exact | learning | slam | grab | last | self | large'),
 ('Topic 3:',
  'pain | reported | amcare | injury | stated | site | incident | left | right | treatment | working | felt | process | lifting | safety | investigation | report | day | time | level | manager | heavy | cart | area | hand | boxes | wrist | continued | case | totes'),
 ('Topic 4:',
  'states | one | aisle | high | knees | reaching | empty | small | response | 6/10 | splitter | pushed | flow | call | approx | sen

In [52]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=16,
                                           passes=5, 
                                           random_state=100,
                                           chunksize=5)#,
#                                            update_every=100,
#                                            #alpha='auto',
#                                            per_word_topics=True)
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus =corpus, dictionary=id2word)
vis

C:\Users\ndsamuel\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.334073 -0.022424       1        1  22.688725
1     -0.298582 -0.064619       2        1  11.270741
0     -0.278106  0.000121       3        1   9.128097
13     0.034859 -0.090917       4        1   8.557519
8      0.011025  0.163556       5        1   6.916580
11     0.057238  0.212101       6        1   5.521571
15    -0.047899  0.209469       7        1   5.294599
6      0.040339 -0.221263       8        1   4.839442
5      0.085384  0.124169       9        1   4.447280
12     0.105356 -0.034905      10        1   3.628577
14     0.111216 -0.016050      11        1   3.406620
10     0.108731 -0.068959      12        1   3.376171
9      0.080016 -0.084263      13        1   3.150300
2      0.108379  0.009241      14        1   2.741548
3      0.122121  0.013020      15        1   2.712706
4      0.093997 -0.128277      16        1   2.319519, topic_info=     Category          Freq          Term         Total  loglift  logprob
term                                                                     
90    Default  21713.000000     associate  21713.000000  30.0000  30.0000
41    Default  26712.000000          pain  26712.000000  29.0000  29.0000
23    Default  10748.000000        injury  10748.000000  28.0000  28.0000
778   Default   4722.000000        pallet   4722.000000  27.0000  27.0000
115   Default  11385.000000      incident  11385.000000  26.0000  26.0000
32    Default   4709.000000       medical   4709.000000  25.0000  25.0000
246   Default   4064.000000          back   4064.000000  24.0000  24.0000
57    Default   3541.000000      shoulder   3541.000000  23.0000  23.0000
374   Default   5303.000000       lifting   5303.000000  22.0000  22.0000
54    Default   7080.000000         right   7080.000000  21.0000  21.0000
386   Default   5106.000000          site   5106.000000  20.0000  20.0000
60    Default   9527.000000        stated   9527.000000  19.0000  19.0000
629   Default   4500.000000           box   4500.000000  18.0000  18.0000
271   Default   3452.000000          pick   3452.000000  17.0000  17.0000
117   Default   3910.000000          knee   3910.000000  16.0000  16.0000
334   Default   7511.000000          left   7511.000000  15.0000  15.0000
147   Default   3306.000000          team   3306.000000  14.0000  14.0000
6     Default   9770.000000        amcare   9770.000000  13.0000  13.0000
264   Default   2709.000000         lower   2709.000000  12.0000  12.0000
52    Default  10339.000000      reported  10339.000000  11.0000  11.0000
326   Default   3060.000000          hand   3060.000000  10.0000  10.0000
73    Default  10800.000000          work  10800.000000   9.0000   9.0000
593   Default   3429.000000         heavy   3429.000000   8.0000   8.0000
27    Default   5929.000000         level   5929.000000   7.0000   7.0000
119   Default   2610.000000          lift   2610.000000   6.0000   6.0000
280   Default   3567.000000  restrictions   3567.000000   5.0000   5.0000
272   Default   2482.000000       picking   2482.000000   4.0000   4.0000
195   Default   2518.000000          item   2518.000000   3.0000   3.0000
308   Default   2421.000000          cart   2421.000000   2.0000   2.0000
348   Default   5400.000000        safety   5400.000000   1.0000   1.0000
...       ...           ...           ...           ...      ...      ...
9     Topic16   1355.930420           arm   1356.761108   3.7632  -2.7306
58    Topic16    532.644104         small    533.474792   3.7623  -3.6650
8     Topic16    443.762787   appropriate    444.593475   3.7619  -3.8475
35    Topic16    441.266296        months    442.096985   3.7619  -3.8532
1784  Topic16    433.848694          pole    434.679382   3.7619  -3.8701
2588  Topic16    422.644012        health    423.474701   3.7618  -3.8963
1383  Topic16    409.139221          slam    409.969910   3.7618  -3.9288
760   Topic16    397.7973

In [48]:
pd.DataFrame(l, columns = 'TopicNo TopicKeyWords'.split()).to_excel('Topics16 Sprain-strain.xlsx', index = False)